# Battle of neighbourhoods WEEK 2 SUBMISSION 

<b> PROBLEM and BACKGROUND</b>

In this exercise we compare New-Your city and Toronto as tourist destinations - to get a view of their boroughs and neighbourhoods, and marked on map their key areas (as mentioned below) with respect to those neighbourhoods and boroughs <br>
<b>Toronto - Downtown Toronto </b> <br>
<b>New York - Manhattan </b> <br>

The analysis will also then compare what are the hotspots for both Downtown Toronoto and Manhattan which could help tourist to locate the most popular places in each neighbourhood by virtue of rankings. This will help them plan their travels to both these places accordinagly and change their travel plan basis which city and area they are in. This will also be indicative of what are the main places/attractions to visit in a particular area of a city for the tourists such as cafes, restaurants, parks etc.


REASON - Tourists most often visits those countries who are rich in heritage. Every city is unique in their own way and has something different to offer. and based on available information, and the comparison (the part of the information) between the two cities tourists can choose to plan their travel or visit a particular place according to their choice.

<b> Data sources and cleaning </b> <br>
For this problem, we will get the services of Foursquare API to explore the data of two cities, in terms of their neighborhoods. The data also include the information about the places around each neighborhood like restaurants, hotels, coffee shops, parks, theaters, art galleries, museums and many more. We selected one Borough from each city to analyze their neighborhoods. Manhattan from New York and Downtown Toronto from Toronto. These objects will be given priority on the basis of foot traffic (activity) in their respective neighborhoods which will help to locate the tourist’s areas and hubs and helps us analyse the similarity or difference between two cities from tourist's perspective

<b>Methodology</b><br>
We will extract the data and analyse/visualise it for both the areas using the same method to arrive at information about each beigbourhood and its hotspot so as to compare the cities<br>

<b> Exploration </b> <br>
the Downtown Toronto data is extracted and cleaned using the data from wikipedia page +.csv file locally stored on system and imported (source - kaggle) and verified using Foursquare API to get the coordinates for neighbourhood exploration <br><br> Manhatattan data for New Yourk Coty is extracted from .csv file locally stored on system and imported (source - kaggle) which is again verified and explored for coordinates using Foursquare API

IMPORTING LIBRARIES

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Visualization
import matplotlib.pyplot
import seaborn as sns
# Too see full dataframe...
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!pip install folium
import folium # map rendering library

print('Libraries imported.')

     |████████████████████████████████| 93 kB 2.8 MB/s  eta 0:00:01
Libraries imported.


<b>Toronto (downtown toronto) Data extraction, cleaning and sorting</b>

In [4]:
# Link To Extract
path='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
# Read File
df_wiki=pd.read_html(path)
#Check the type
type(df_wiki)
# Call the position where the table is stored
neighborhood=df_wiki[0]
# Rename the Columns
neighborhood.rename(columns={'Postal Code':'Postcode', 1: 'Borough', 2: 'Neighborhood'}, inplace=True)
# Eliminate the first row
neighborhood=neighborhood.drop([0])
# Eliminate "Not assigned", categorical values from "Borough" Column
neighborhood=neighborhood[neighborhood.Borough !='Not assigned']
# Making DataFrame
neighborhood=pd.DataFrame(neighborhood)
# Merging rows with same Postcode
neighborhood.set_index(['Postcode','Borough'],inplace=True)
merge_result = neighborhood.groupby(level=['Postcode','Borough'], sort=False).agg( ','.join)
# Setting the index
serial_wise=merge_result.reset_index()
# Assign the 'Borough' column value to 'Neighborhood' where 'Not assigned' occurs
serial_wise.loc[4, 'Neighborhood']='Queen\'s Park'

# Showing the Data Frame
df=pd.DataFrame(serial_wise)
df.head()

,Postcode,Borough,Neighbourhood,Neighborhood
0,M3A,North York,Parkwoods,NaN
1,M4A,North York,Victoria Village,NaN
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",NaN
3,M6A,North York,"Lawrence Manor, Lawrence Heights",NaN
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",Queen's Park


In [9]:
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_704f77be6a9a4570b77bc074b52c04e5 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='V-QTQ6tPeuaBYIoQXBbHF3AiRbeanw_Eebr281S-Fcgv',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_704f77be6a9a4570b77bc074b52c04e5.get_object(Bucket='capstoneproject-donotdelete-pr-4rpxl4ca1lj1ju',Key='neigbors_geographical.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df1 = pd.read_csv(body)
df1.head()
# Change the Postal Code to Postcode
df1.rename(columns={'Postal Code':'Postcode'},inplace=True)
#Cancatenation
frames=[df,df1]
frames=pd.concat(frames, axis=1, sort=False)
# Merging the two columns on 'Postcode'
merge_columns=pd.merge(df, df1, left_on='Postcode', right_on='Postcode')
# Save the Data Frame
merge_columns.to_csv('neigbors_geographical.csv')
merge_columns.head()

,Postcode,Borough_x,Neighbourhood,Neighborhood_x,Unnamed: 0,Borough_y,Neighborhood_y,Latitude,Longitude
0,M3A,North York,Parkwoods,NaN,0,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,NaN,1,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",NaN,2,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",NaN,3,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",Queen's Park,4,Queen's Park,Queen's Park,43.662301,-79.389494


In [11]:
downtown_toronto_data = merge_columns[merge_columns['Borough_x'] == 'Downtown Toronto'].reset_index(drop=True)
# eliminate 'Postcode' column
downtown_toronto_data=downtown_toronto_data.drop(['Postcode'], axis=1)
downtown_toronto_data.head()

,Borough_x,Neighbourhood,Neighborhood_x,Unnamed: 0,Borough_y,Neighborhood_y,Latitude,Longitude
0,Downtown Toronto,"Regent Park, Harbourfront",NaN,2,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
1,Downtown Toronto,"Queen's Park, Ontario Provincial Government",Queen's Park,4,Queen's Park,Queen's Park,43.662301,-79.389494
2,Downtown Toronto,"Garden District, Ryerson",NaN,9,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937
3,Downtown Toronto,St. James Town,NaN,15,Downtown Toronto,St. James Town,43.651494,-79.375418
4,Downtown Toronto,Berczy Park,NaN,20,Downtown Toronto,Berczy Park,43.644771,-79.373306


<b>New York (Manhattan) Data extraction, cleaning and sorting</b>

In [18]:

body = client_704f77be6a9a4570b77bc074b52c04e5.get_object(Bucket='capstoneproject-donotdelete-pr-4rpxl4ca1lj1ju',Key='bars_with_hood.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

neighborhoods = pd.read_csv(body)
neighborhoods.head()
# And make sure that the dataset has all 5 boroughs and 306 neighborhoods.

print('The dataframe has {} borough and {} neighborhood.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

neighborhoods.head()

The dataframe has 7 borough and 2580 neighborhood.


,Unnamed: 0,index_right,Location Type,Incident Zip,City,Borough,Latitude,Longitude,num_calls,neighborhood
0,0,122,Club/Bar/Restaurant,10308.0,STATEN ISLAND,STATEN ISLAND,40.544096,-74.141155,40.0,Great Kills
1,1,122,Club/Bar/Restaurant,10308.0,STATEN ISLAND,STATEN ISLAND,40.544209,-74.141040,21.0,Great Kills
2,2,122,Club/Bar/Restaurant,10308.0,STATEN ISLAND,STATEN ISLAND,40.544528,-74.140673,21.0,Great Kills
3,3,122,Club/Bar/Restaurant,10308.0,STATEN ISLAND,STATEN ISLAND,40.544308,-74.140907,34.0,Great Kills
4,4,126,Club/Bar/Restaurant,10012.0,NEW YORK,MANHATTAN,40.729793,-73.998842,18.0,Greenwich Village


In [20]:
# Creating new Dataframe manhattan_data
manhattan_data = neighborhoods[neighborhoods['Borough'] == 'MANHATTAN'].reset_index(drop=True)
manhattan_data.head()

,Unnamed: 0,index_right,Location Type,Incident Zip,City,Borough,Latitude,Longitude,num_calls,neighborhood
0,4,126,Club/Bar/Restaurant,10012.0,NEW YORK,MANHATTAN,40.729793,-73.998842,18.0,Greenwich Village
1,5,126,Club/Bar/Restaurant,10012.0,NEW YORK,MANHATTAN,40.727076,-74.000184,40.0,Greenwich Village
2,6,126,Club/Bar/Restaurant,10012.0,NEW YORK,MANHATTAN,40.729519,-73.999812,120.0,Greenwich Village
3,7,126,Club/Bar/Restaurant,10012.0,NEW YORK,MANHATTAN,40.729750,-73.999614,41.0,Greenwich Village
4,8,126,Club/Bar/Restaurant,10012.0,NEW YORK,MANHATTAN,40.729338,-74.001032,99.0,Greenwich Village


<b> Mapping Toronto and Manahattan Neighbourhoods </b><br> visualizing the selected borough neighborhoods so that we can get an idea or confirmation regarding the coordinates of that Borough.

In [36]:
# get the geographical coordinates of Downtown Toronto
address = 'Downtown Toronto, ON, Canada'

geolocator = Nominatim(user_agent="http")
location = geolocator.geocode(address)
latitude_downtown_toronto = location.latitude
longitude_downtown_toronto = location.longitude
print("Downtown Toronto","latitude",latitude_downtown_toronto, "& " "longitude" ,longitude_downtown_toronto)

Downtown Toronto latitude 43.6563221 & longitude -79.3809161


In [37]:
# Let's get the geographical coordinates of Manhattan.
address = 'Manhattan, NY'

geolocator = Nominatim(user_agent="http")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Manhattan are 40.7896239, -73.9598939.


In [39]:
# create map of Downtown Toronto using latitude and longitude values
map_downtown_toronto = folium.Map(location=[latitude_downtown_toronto,longitude_downtown_toronto], zoom_start=11)

# add markers to map
for lat, lng, label in zip(downtown_toronto_data['Latitude'], downtown_toronto_data['Longitude'], downtown_toronto_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_downtown_toronto)  
    
map_downtown_toronto

In [40]:
from folium import plugins
# create map of Downtown Toronto using latitude and longitude values
map_downtown_toronto = folium.Map(location=[latitude_downtown_toronto,longitude_downtown_toronto], zoom_start=11)
# instantiate a mark cluster object for the incidents in the dataframe
incidents = plugins.MarkerCluster().add_to(map_downtown_toronto)
# add markers to map
for lat, lng, label in zip(downtown_toronto_data['Latitude'], downtown_toronto_data['Longitude'], downtown_toronto_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(incidents)  
    
map_downtown_toronto

In [42]:
# let's visualization of Manhattan and the neighborhoods in it.
# create map of Manhattan using latitude and longitude values
map_manhattan = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_manhattan)  
    
map_manhattan

In [43]:
# create map of Manhattan using latitude and longitude values
map_manhattan = folium.Map(location=[latitude, longitude], zoom_start=11)

grouping = plugins.MarkerCluster().add_to(map_manhattan)

# add markers to map
for lat, lng, label in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(grouping)  
    
map_manhattan


<b>Foursquare API

In [53]:
# Define Foursquare Credentials and Version ***sample taken
CLIENT_ID = 'HRMBKZUASN1NWO005IQK4TGG15UVEY5GCLJCYXHXW0VDP00K' # your Foursquare ID
CLIENT_SECRET = 'JSXFO23NR2OMICQSZRFQYDAZG1GMNRALXXACAFVNF5CGAM4C' # your Foursquare Secret
VERSION = '20180604'
limit = 20


# ANALYSIS

We calculate mean of the frequency of occurrence of each category (using one shot encoding) and picked top ten venues on that basis for each neighborhood. It means the top venues are showing the foot traffic or the more visited places.<br>


<b> DOWNTOWN TORONTO - Analysis

In [54]:
# Let's create a function to repeat the process to all the neighborhoods in Toronto
def getNearbyVenues(names, latitudes,longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names,latitudes,longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [56]:
# Write the code to run the above function on each neighborhood and create a new dataframe called toronto_venues.
downtown_toronto_venues = getNearbyVenues(names=downtown_toronto_data['Neighbourhood'],
                                   latitudes=downtown_toronto_data['Latitude'],
                                   longitudes=downtown_toronto_data['Longitude'],
                                  )

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Rosedale
Stn A PO Boxes
St. James Town, Cabbagetown
First Canadian Place, Underground city
Church and Wellesley


In [58]:
# one hot encoding
downtown_toronto_onehot = pd.get_dummies(downtown_toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
downtown_toronto_onehot['Neighborhood'] = downtown_toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [downtown_toronto_onehot.columns[-1]] + list(downtown_toronto_onehot.columns[:-1])
downtown_toronto_onehot = downtown_toronto_onehot[fixed_columns]


# Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category
downtown_toronto_grouped = downtown_toronto_onehot.groupby('Neighborhood').mean().reset_index()

In [59]:
# Let's put that into a pandas dataframe
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [60]:
# Now let's create the new dataframe and display the top 10 venues for each neighborhood.
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = downtown_toronto_grouped['Neighborhood']

for ind in np.arange(downtown_toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(downtown_toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Beer Bar,Farmers Market,Park,Seafood Restaurant,Museum,Cocktail Bar,Coffee Shop,Concert Hall,Liquor Store,Bistro
1,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Lounge,Airport Service,Airport Terminal,Sculpture Garden,Harbor / Marina,Boat or Ferry,Coffee Shop,Bar,Rental Car Location,Boutique
2,Central Bay Street,Coffee Shop,Gastropub,Spa,Modern European Restaurant,Miscellaneous Shop,Café,Middle Eastern Restaurant,Park,Bubble Tea Shop,Ramen Restaurant
3,Christie,Grocery Store,Café,Park,Italian Restaurant,Coffee Shop,Candy Store,Restaurant,Baby Store,Athletics & Sports,Nightclub
4,Church and Wellesley,Park,Bubble Tea Shop,Ramen Restaurant,Salon / Barbershop,Juice Bar,Creperie,Dance Studio,Bookstore,Diner,Breakfast Spot
5,"Commerce Court, Victoria Hotel",Café,Gastropub,Coffee Shop,Restaurant,Museum,Bakery,Gym / Fitness Center,Sandwich Place,American Restaurant,Gluten-free Restaurant
6,"First Canadian Place, Underground city",Café,Restaurant,Coffee Shop,Gastropub,Steakhouse,Pizza Place,Pub,Seafood Restaurant,Speakeasy,Asian Restaurant
7,"Garden District, Ryerson",Café,Art Gallery,Electronics Store,Ramen Restaurant,Sandwich Place,Music Venue,College Rec Center,Diner,Mexican Restaurant,Plaza
8,"Harbourfront East, Union Station, Toronto Islands",Park,Hotel,Plaza,New American Restaurant,Lake,Salad Place,Japanese Restaurant,Deli / Bodega,Skating Rink,Ice Cream Shop
9,"Kensington Market, Chinatown, Grange Park",Café,Wine Bar,Cocktail Bar,Vietnamese Restaurant,Gourmet Shop,Fish Market,Farmers Market,Dessert Shop,Coffee Shop,Cheese Shop


<b>New York - MANHATTAN - Analysis

In [67]:
# Let's create a function to repeat the same process to all the neighborhoods in Manhattan
def getNearbyVenues(names, latitudes, longitudes, radius=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [68]:
# Now write the code to run the above function on each neighborhood and create a new dataframe called manhattan_venues
manhattan_venues = getNearbyVenues(names=manhattan_data['neighborhood'],
                                   latitudes=manhattan_data['Latitude'],
                                   longitudes=manhattan_data['Longitude'],
                                  )

In [71]:
# one hot encoding
manhattan_onehot = pd.get_dummies(manhattan_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
manhattan_onehot['Neighborhood'] = manhattan_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

# Set Index
manhattan_grouped = manhattan_onehot.groupby('Neighborhood').mean().reset_index()

In [72]:
# Now let's create the new dataframe and display the top 10 venues for each neighborhood.
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = manhattan_grouped['Neighborhood']

for ind in np.arange(manhattan_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(manhattan_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central Park,Clothing Store,Playground,Italian Restaurant,Sandwich Place,Dog Run,Park,Outdoor Sculpture,Steakhouse,American Restaurant,Hotel
1,Chelsea,Nightclub,Hotel,Coffee Shop,Italian Restaurant,Gym / Fitness Center,Café,Cocktail Bar,Juice Bar,Theater,Yoga Studio
2,Chinatown,Chinese Restaurant,Bar,American Restaurant,Bakery,Cocktail Bar,Noodle House,Sandwich Place,Boutique,Dumpling Restaurant,Dim Sum Restaurant
3,East Harlem,Mexican Restaurant,Pizza Place,Bakery,Taco Place,Deli / Bodega,Café,Cosmetics Shop,Thai Restaurant,Donut Shop,Italian Restaurant
4,East Village,Bar,Coffee Shop,Cocktail Bar,Italian Restaurant,Pizza Place,Vietnamese Restaurant,Japanese Restaurant,Mexican Restaurant,Vegetarian / Vegan Restaurant,Ice Cream Shop
5,Financial District,Pizza Place,Coffee Shop,Pet Store,Mexican Restaurant,Sandwich Place,Café,Playground,Hotel,Burger Joint,Greek Restaurant
6,Flatiron District,Spa,Japanese Restaurant,Furniture / Home Store,Café,Gym / Fitness Center,Cycle Studio,Coffee Shop,Korean Restaurant,Italian Restaurant,American Restaurant
7,Gramercy,American Restaurant,Italian Restaurant,Bar,Café,Food Truck,Pizza Place,Pub,Wine Bar,New American Restaurant,Cheese Shop
8,Greenwich Village,Italian Restaurant,Indian Restaurant,Bar,Chinese Restaurant,Vietnamese Restaurant,Coffee Shop,Wine Bar,Sushi Restaurant,Café,Bubble Tea Shop
9,Harlem,African Restaurant,Mexican Restaurant,Deli / Bodega,Cosmetics Shop,Bar,Seafood Restaurant,Southern / Soul Food Restaurant,Coffee Shop,Grocery Store,Sandwich Place


<b> Result </b> <br> After clustering the data of the respective neighborhoods, both cities (Boroughs) venues which can be explored and attract the tourists are much similar in features like  food places, clubs, museums, parks etc

<b>Observations and Recommensation </b> <br> The number of options to explore as tourist attractions are more in Manhattan than in Downtown Toronto <br> Additionally Toronto has highest footfall in cafes as the first choice venue which shows the local and tourist behaviour in downtown toronto in comparison to Manhattan where people mostly visit restaurants and bars for first choice in most of neighbourhoods 

<b> CONCLUSION</b> <br> The downtown Toronto and Manhattan neighborhoods have a lot of similar venues but vary in size and number of places to explore 

<b> Future Direction </b> <br> the Analysis can be expanding to analyse different clusters in these neighbourhoods based on the venues and can be then mapped to determine the discriminating venue categories that distinguish each cluster.<br> Furthermore the analysis can be expanded to other boroughs and neighbourhoods.